In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
plt.style.use('fivethirtyeight')

from datetime import datetime

In [ ]:
from sklearn import preprocessing
from sklearn import svm
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE,SMOTENC
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, f1_score
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold

In [ ]:
#Affichage des 4 dataframes séparés 

In [ ]:
lieux = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/8a4935aa-38cd-43af-bf10-0209d6d17434',sep= ";")
usagers = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/ba5a1956-7e82-41b7-a602-89d7dd484d7a',sep = ';')
vehicules = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/0bb5953a-25d8-46f8-8c25-b5c2f5ba905e', sep= ";")
caracteristiques = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/85cfdc0c-23e4-4674-9bcd-79a970d7269b',sep = ";")

In [ ]:
lieux.isnull().sum()

In [ ]:
caracteristiques.isnull().sum()

In [ ]:
vehicules.isnull().sum()

In [ ]:
usagers.isnull().sum()

In [ ]:
usagers["Num_Acc"].value_counts()

In [ ]:
df1 = pd.merge(lieux,caracteristiques)
df1

In [ ]:
df2 = pd.merge(usagers,vehicules, how = 'left')
df2

In [ ]:
df = pd.merge(df2,df1,how = 'left')
df

In [ ]:
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_percentages = df.isna().sum().sort_values(ascending=False)/len(df)
#missing_percentages, missing_values

In [ ]:
df.dtypes

On sait que les colonnes lartpc, occutc, v2 correspondsant respectivement à la largeur du terre-plein central (TPC) s'il existe (en m, au nombre d’occupants dans le transport en commun) et à la lettre indice alphanumérique de la route comportent plus de 90% de valeurs manquantes NaN donc on peut retirer ses colonnes immédiatement du dataset.  

In [ ]:
df2 = df.copy()

In [ ]:
df.drop(['lartpc','occutc','v2'],axis = 1,inplace = True)

On enlève aussi les colonnes Num_Acc, id_vehicule et Num_Veh qui sont des clés que nous avons utilisé pour merge nos DataFrame.

In [ ]:
df.drop(['Num_Acc','id_vehicule','num_veh'],axis = 1,inplace = True)

On peut aussi enlever les colonnes pr, pr1, voie, v1 et senc correspondant respectivement au numéro du PR de rattachement (numéro de la borne amont), à la distance en mètres au PR, au numéro de la route, à l'indice numérique du numéro de route et au sens de circulation car ils ne sont pas utiles pour prédire la gravité d'un accident. 

In [ ]:
df.drop(['pr','pr1','voie','v1','senc'],axis = 1,inplace = True)

In [ ]:
df.columns

In [ ]:
# EXPLIQUER LE BUT DE CETTE CELLULE
import re 

def cleanData(data):
    data = re.sub(',','.',data) 
    return data

df['lat2']=df['lat'].apply(cleanData)
df['long2']=df['long'].apply(cleanData)

df["lat2"] = pd.to_numeric(df["lat2"])
df["long2"] = pd.to_numeric(df["long2"])

df.drop(['lat','long'],axis = 1,inplace = True)

In [ ]:
# La colonne actp contenait des nombres convertis en string donc on la repasse en float

def cleanData2(data):
    data = re.sub(',','.',data) 
    data = re.sub('B','-1',data)
    data = re.sub('A','9',data)
    return data

df['larrout']=df['larrout'].apply(cleanData2)
df['larrout'] = pd.to_numeric(df['larrout'])

df['actp']=df['actp'].apply(cleanData2)
df['actp'] = pd.to_numeric(df['actp'])

In [ ]:
def time(data):
    return datetime.strptime(data, "%H:%M").time()

df['hrmn']=df['hrmn'].apply(time)

In [ ]:
# Calculer le nombre de -1 dans chaque colonne car ces valeurs signifient : informations non renseignées

def renseignement():
    L = []
    for col in list(df):
        L.append(len(df[df[col]==-1])/df.shape[0])
    return L

liste = renseignement()

# Indices des colonnes contenant plus de 20% de valeur -1 correspondant à des valeurs non renseignées 

def indices(liste):
    indices = []
    longueur = len(liste)
    for i in range(longueur):
        taux = liste[i]
        if taux > 0.2:
            indices.append(i)
    return indices
            
indices = indices(liste) 
df.columns[indices]

In [ ]:
# On retire donc ces 6 colonnes de notre dataset
df.drop(df.columns[indices], axis = 1, inplace = True)

Ces indices correspondent aux colonnes secu2, secu3, locp, actp et etatp et larrout correspondant respectivement aux renseignements du caractère indiquant la présence et l’utilisation de l’équipement de sécurité pour secu2 et 3, à la localisation du piéton pour locp, à une variable permettant de préciser si le piéton accidenté était seul ou non pour etatp et à la largeur de la chaussée affectée à la circulation des véhicules pour larrout. 
On peut donc supprimer ces colonnes de notre dataset. 

In [ ]:
indemne = len(df[df.grav == 1])
bléssé_léger = len(df[df.grav == 4])
bléssé_hospi = len(df[df.grav == 3])
tué = len(df[df.grav == 2])

indexNames = df[df['grav'] == -1 ].index
df.drop(indexNames ,inplace=True)

gravité = [indemne, bléssé_léger, bléssé_hospi, tué]

# Data visualisation et statistiques descriptives 

In [ ]:
df.info()

Seule la colonne an_nais possède des valeurs manquantes (3007). Nous nous en occuperons plus tard lors de la modélisation.

In [ ]:
df.describe()

On peut voir qu'une personne agée de 109 ans et un bébé de moins d'un an ont été impliqués dans des accidents en 2021.

In [ ]:
df.loc[df['an_nais'] == 1912]

Cette personne de 109 ans a donc été hospitalisée. Elle était sur le siège passager lors de l'accident, sur une route limitée à 50 km/h, en pleine nuit avec éclairage public allumé.  

Nous remarquons aussi que la vitesse maximale est de 901 km/h. Nous pensons à une erreur de frappe, nous corrigeons donc cela en 90 km/h. Plus généralement, nous regardons la distribution des valeurs de vitesse. Nous changeons 300 en 30, de 500 à 520 en 50 et 700 et 770 en 70.

In [ ]:
np.unique(df['vma'], return_counts = True)

In [ ]:
abus = [300,500,501,502,520,700,770,900,901]

In [ ]:
for i in range(len(abus)):
    e = abus[i]
    chiffre = e//100
    new_vma = float(str(chiffre)+str(0))
    req_Index = list(np.where(df["vma"] == e))
    for j in req_Index[0]:
        df.iloc[j,10] = new_vma

In [ ]:
np.unique(df['vma'], return_counts = True)

In [ ]:
def cat(annee):
    if annee >= 1996:
        return "moins de 25 ans"
    elif annee > 1961:
        return "entre 25 et 60 ans"
    else :
        return "plus de 60 ans"

In [ ]:
df3 = df.copy()
df3['category_age'] = df['an_nais'].apply(cat)

In [ ]:
sns.countplot(y='grav',data=df3, hue ='category_age').set(title="distribution de la gravité en fonction de l'âge")

La catégorie avec le plus de conducteurs actifs est donc la plus représentée dans chacune des classes de gravité.

Longitude de la France métropolitaine est entre -4° et 8°, sa latitude entre 42° et 51°. La base de données ne se restreint donc pas seulement à la France métropolitaine, elle comprend aussi l'Outre-Mer français.

## Villes

In [ ]:
communes = df2.com.unique()
len(communes)

Les accidents reportés dans cette base de données ne se sont produits que dans moins de 12000 communes, sur plus de 36000 en France selon l'INSEE.

In [ ]:
plt.figure(figsize=(10,5))
df2.dep.value_counts().sort_values(ascending=False)[:10].plot(kind='bar').set(title = "distribution du nombre d'accidents en fonction du département")

Sans surprise, la ville de Paris contient le plus grand nombre d'accidents routiers. Cela n'implique pas directement que conduire à Paris augmente la probabilité d'avoir un accident, il faut prendre en compte le nombre de voitures circulant dans la capitale.

In [ ]:
paris =df2[df2['dep']=='75']
plt.figure(figsize=(10,15))
paris.com.value_counts().plot(kind='pie',autopct='%1.1f%%').set(title = "répartition des d'accidents parisiens en fonction de l'arrondissement")

centre_circle = plt.Circle((0, 0), 0.70, fc='white')
fig = plt.gcf()
  
# Adding Circle in Pie chart
fig.gca().add_artist(centre_circle)
  

Le 16e arrondissement de Paris est celui qui comporte le plus d'accidents de la route 

In [ ]:
### Brouillon API noms de villes ###

In [ ]:
code_commune="85048"
api_root="https://geo.api.gouv.fr/communes"
url_api = f"{api_root}?code="+ f"{code_commune}"  

import requests
import pandas as pd

req = requests.get(url_api)
wb = req.json()

df3 = pd.json_normalize(wb)

df3

In [ ]:
df2

In [ ]:
api_root="https://geo.api.gouv.fr/communes"  

import requests
import pandas as pd

req = requests.get(api_root)
wb = req.json()

df3 = pd.json_normalize(wb)

df3=df3[['nom','code']]
df3.columns = ['nom_villes', 'com']
df3

In [ ]:
df4 = pd.merge(df2,df3,how = 'left')
df4

import time
t1 = time.time()

for i in range(len(df4)):
    if df4['com'][i] in list(df2[df2['dep']=='75'].com):
        df4['nom_villes'][i]='Paris'
    

print(time.time()- t1)

    

In [ ]:
#df4.isnull().sum().sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(10,5))
df4.nom_villes.value_counts().sort_values(ascending=False)[:10].plot(kind='bar').set(title = "villes avec le plus d'accidents")

In [ ]:
def get_city_from_url(code):
    try : 
        code_commune = code
        api_root="https://geo.api.gouv.fr/communes"
        url_api = f"{api_root}?code="+ f"{code_commune}"  
        req = requests.get(url_api)
        wb = req.json()

        a = pd.json_normalize(wb)
        return a['nom'][0]
    except : 
        return np.NaN 


df2['nom_villes']=df2['com'].apply(get_city_from_url)

df

In [ ]:
api_root="https://geo.api.gouv.fr/communes"
req = requests.get(api_root)
wb = req.json()

df = pd.json_normalize(wb)

df[df['nom']=='Paris']



In [ ]:
### FIN BROUILLON API ###

In [ ]:
df_sample = df[:3000]
df_sample

In [ ]:
import folium

palette = sns.color_palette("coolwarm", 8)

def interactive_map(df):

    # convert in number
    df_sample['color'] = [int(df_sample.iloc[i]['grav']) for i in range(len(df_sample))]

    df_sample['color'] = [palette.as_hex()[x] for x in df_sample['color']]


    center = df_sample[['lat2', 'long2']].mean().values.tolist()
    sw = df_sample[['lat2', 'long2']].min().values.tolist()
    ne = df_sample[['lat2', 'long2']].max().values.tolist()

    m = folium.Map(location = center, tiles='Stamen Toner')

    # I can add marker one by one on the map
    for i in range(0,len(df)):

        folium.Marker([df_sample.iloc[i]['lat2'], df_sample.iloc[i]['long2']],
                    popup=f"Luminosité: {df_sample.iloc[i]['lum']}, <br>gravité: {df_sample.iloc[i]['grav']}, <br>vma: {df_sample.iloc[i]['vma']}",
                    icon=folium.Icon(icon='car', prefix='fa')).add_to(m)

    m.fit_bounds([sw, ne])

    return m

m = interactive_map(df_sample)

In [ ]:
m

In [ ]:
pip install geojson

In [ ]:
import geopandas as gpd
import json
import geojson
departements = gpd.read_file('accident-route/departements-france.geojson')
departements.head()

In [ ]:
dep_accident = df2.dep.value_counts().to_frame().reset_index()
dep_accident["échelle d'incidence"]=np.log10(dep_accident['dep'])
dep_accident

In [ ]:
import plotly.express as px

fig1 = px.choropleth_mapbox(dep_accident, locations = 'index',
                            geojson= departements,
                            color="échelle d'incidence",
                            color_continuous_scale=["green","orange","red"],
                            range_color=[1,4],
                            hover_name='index',
                            hover_data=['dep'],
                            title="Carte de répartition des accidents en France par département",
                            mapbox_style="open-street-map",
                            center= {'lat':46, 'lon':2},
                            zoom =3.5, 
                            opacity= 0.6)

fig1.show()

In [ ]:
df_samp = df.sample(int(len(df)*0.01))

df_samp 
fig2 = px.density_mapbox(df_samp, lat='lat2',lon='long2', radius=10, center=dict(lat=48.9, lon=2.4), zoom=4,
                         mapbox_style="stamen-watercolor")
fig2.show()



# OU avec tous les accidents : moins lisible 
# df3 = df.copy()
# fig2 = px.density_mapbox(df3, lat='lat2',lon='long2', radius=10,
#                       center=dict(lat=48.9, lon=2.4), zoom=4,
#                       mapbox_style="stamen-watercolor")
# fig2.show()

## Heures d'accident 

In [ ]:
def time(data):
    return datetime.strptime(data, "%H:%M").time()

df2['hrmn']=df2['hrmn'].apply(time)

In [ ]:
df.hrmn = pd.to_datetime(df.hrmn)
df.hrmn.dt.hour

In [ ]:
sns.histplot(df.hrmn.dt.hour,bins=24)

In [ ]:
sns.distplot(df.hrmn.dt.hour,bins=24)

On observe d'un pic d'accidents vers 17h (heure de pointe donc plus de monde sur les routes donc plus d'accidents). 

### Jours de la semaine 

In [ ]:
sns.histplot(df.jour,bins = 31).set(title = "distribution du nombre d'accidents en fonction du jour")

### Mois de l'année 

In [ ]:
sns.histplot(df.mois,bins = 12).set(title = "distribution du nombre d'accidents en fonction du mois")

In [ ]:
df.mois.dt.month

In [ ]:
timestamp = datetime.timestamp((df2.hrmn)[0])

In [ ]:
sns.histplot(df2.hrmn,bins=24)

In [ ]:
df.drop(df.loc[df['grav']==-1].index, inplace=True)
df['grav'].value_counts()

In [ ]:
df.grav.value_counts().plot(kind='bar',xlabel='Niveau de gravité',ylabel='Nombre de personnes').set(title = "classes de gravité")

In [ ]:
df.groupby('lum')['grav'].value_counts().plot

In [ ]:
sns.countplot(y='grav',data=df,hue='lum').set(title = "distribution de la luminosité en fonction de la gravité")

Luminosité = 1 correspond au plein jour. Quelque soit la gravité, il y a beaucoup plus d'accidents le jour dû au nombre de voitures. Pour la gravité = 2 qui correspond à tué, on remarque que cela s'équilibre plus avec lum = 3 qui correspond à la nuit sans éclairage public. Cela semble cohérent : les conditions de circulation sont plus dangereuses.

In [ ]:
sns.countplot(y='grav',data=df,hue='sexe').set(title = "distribution du sexe en fonction de la gravité")

Quelque soit la gravité, il y a beaucoup plus d'hommes impliqués dans des accidents de voiture que de femmes.

In [ ]:
def vitesse(speed):
    if speed < 50 : 
        return " moins de 50 km/h"
    elif speed < 80 :
        return "entre 50 et 80 km/h"
    elif speed < 130:
        return "entre 80 et 130 km/h"
    else:
        return "plus de 130 km/h"

In [ ]:
dfvitesse = df.copy()
dfvitesse['category_speed'] = df['vma'].apply(vitesse)

In [ ]:
sns.displot(y='category_speed',data=dfvitesse,hue='grav').set(title="distribution de la gravité en fonction de la vitesse")

La grande majorité des accidents ont lieu entre 50 et 80 km/h. Cependant, les accidents les plus mortels se plus produisent à haute vitesse.

In [ ]:
sns.histplot(df.Start_Time.dt.dayofweek,bins=7)
plt.xlabel('Weekdays')

In [ ]:
sns.scatterplot(x=df.long, y=df.lat)

In [ ]:
import re 

def cleanData(data):
    data = re.sub(',','.',data) 
    return data

df['lat2']=df['lat'].apply(cleanData)
df['long2']=df['long'].apply(cleanData)

df["lat2"] = pd.to_numeric(df["lat2"])
df["long2"] = pd.to_numeric(df["long2"])

In [ ]:
def cleanData(data):
    data = re.sub(',','.',data) 
    return data

In [ ]:
df['lat2']=df['lat'].apply(cleanData)
df['long2']=df['long'].apply(cleanData)

In [ ]:
df["lat2"] = pd.to_numeric(df["lat2"])
df["long2"] = pd.to_numeric(df["long2"])

In [ ]:
sns.scatterplot(x=df.long2, y=df.lat2)

In [ ]:
pip install folium

In [ ]:
import folium 

In [ ]:
from folium.plugins import HeatMap

In [ ]:
liste = list(zip(list(df.lat2), list(df.long2)))

In [ ]:
map = folium.Map()
HeatMap(liste).add_to(map)
map

## DATA SELECTION

In [ ]:
df.var().sort_values()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df[['agg','vma','grav','sexe', 'catu', 'plan','prof', 'vosp', 'surf',
                'circ','motor','atm','lum','place','lat2','long2',
                'catv']].corr(),annot=True,linewidth=.5,cmap='YlGnBu').set(title = "matrice de corrélation de plusieurs features")

Les colonnes prof, catu, plan, vosp et circ correspondant respectivement au profil en long décrivant la déclivité de la route à l'endroit de l'accident, la catégorie d'usager, le tracé en plan, l’existence d’une voie réservée, indépendamment du fait que l’accident ait lieu ou non sur cette voie, le régime de circulation ont des variances très faibles, entre 0.1 et 1, pour des variables catégorielles non binaires, donc on peut les retirer des features aidant à la prédiction de notre modèle. 

De plus, la heatmap montre que 'catu' et 'place' ont une corrélation de 0.89 donc on peut garder uniquement 'place' en feature. De même, 'surf' et 'atm' sont corrélés à 0.25 donc on ne peut en garder qu'un seul en feature. Aussi, 'agg' et 'vma'(vitesse maximale autorisée sur la portion de route correspondante) sont très corrélés négatvement (-0.64) donc on peut ne garder que vma en feature. De plus, 'vosp' est corrélé à 'agg' corrélé à 'vma' donc on peut les enlever aussi. 'Circ' a une corrélation significative de 0.19 avec 'vma' donc on peut aussi le retirer du datset pour la prédiction. 

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df[['vma','motor','atm','lum','place','obs','obsm','int','situ',
                'choc']].corr(),annot=True,linewidth=.5,cmap='YlGnBu').set(title = "matrice de corrélation de plusieurs features")

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df[['place','grav', 'sexe', 'an_nais', 'trajet', 'secu1',
       'catv', 'obs', 'obsm', 'choc', 'manv', 'motor', 'catr','nbv','infra','situ',
                'vma']].corr(),annot=True,linewidth=.5,cmap='YlGnBu').set(title = "matrice de corrélation de plusieurs features")

'obs' et 'obsm' ont une corrélation de -0.3 ce qui est élevé donc on n'en garde qu'un seul en feature ('obsm'). 
'catr' et 'vma' ont une corrélation de -0.47 ce qui est élevé donc on n'en garde qu'un seul en feature ('vma').
'situ' et 'obsm' ont une corrélation de -0.16 ce qui est élevé donc on n'en garde qu'un seul en feature ('obsm').
'nbv' et 'vma' ont une corrélation de 0.33 ce qui est élevé donc on n'en garde qu'un seul en feature ('vma').

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df[['place','grav', 'sexe','trajet', 'secu1','int',
       'catv','obsm', 'choc', 'manv','infra','vma']].corr(),annot=True,linewidth=.5,cmap='YlGnBu').set(title = "matrice de corrélation de plusieurs features")

'infra', 'trajet' et 'manv' peuvent être retirés car ils sont corrélés à d'autres variables utilisés déjà en feature et sont ainsi peu utiles. 

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df[['place','grav','sexe','int','an_nais','secu1',
       'catv','obsm', 'choc', 'vma','jour', 'mois', 'hrmn', 'lum', 'dep', 'com', 'atm',
       'col', 'adr', 'lat2', 'long2']].corr(),annot=True,linewidth=.5,cmap='YlGnBu').set(title = "matrice de corrélation de plusieurs features")

'col' et 'obsm' ont une corrélation négative de -0.4 ce qui est élevé donc on n'en garde qu'un seul en feature ('obsm'). 
De plus, 'dep' et 'com' ne sont pas utiles puisqu'on dispose de la latitude et de la longitude. 
Les variables 'jour' et 'mois' ne seront pas utiles pour la prédiction, n'influant pas sur la gravité d'un accident.

On supprime directement la colonne an : nous n'avons que des données de 2021.

In [ ]:
df.drop(['trajet','catu',
       'obs', 'catr', 'circ', 'nbv', 'vosp',
       'prof', 'plan', 'surf', 'infra', 'situ', 'jour', 'mois', 'an',
       'hrmn', 'dep', 'com', 'agg', 'col', 'adr'], axis = 1, inplace = True)

## MODELISATION

In [ ]:
dfmodel = df.copy()

In [ ]:
dfmodel.columns

In [ ]:
labels, counts = np.unique(dfmodel['grav'], return_counts=True)
plt.bar(labels, counts)
plt.gca().set_xticks(labels)
plt.title('distribution des classes de gravité')
plt.show()

On rappelle que les classes sont  : 1 = indemne, 2 = tué, 3 = hospitalisé, 4 = bléssé léger.
On remarque un fort déséquilibrage des classes visées pour la prédiction. En premier lieu, nous regroupons les tués et hospitalisés en une seule classe.

In [ ]:
dfmodel['grav'] =  dfmodel['grav'].apply(lambda x: 2 if x == 3 else x) # fusion des classes 2 et 3.
dfmodel['grav'] =  dfmodel['grav'].apply(lambda x: 3 if x == 4 else x) # classe 4 devient la nouvelle classe 3
dfmodel['grav'] =  dfmodel['grav'].apply(lambda x: x-1) # première classe doit être 0

In [ ]:
labels2, counts2 = np.unique(dfmodel['grav'], return_counts=True)
plt.bar(labels2, counts2)
plt.gca().set_xticks(labels2)
plt.title('distribution des classes de gravité')
plt.show()

Nous voulons ré-équilibrer les classes de gravité pour éviter un biais de prédiction. Pour cela, nous utilisons un algorithme d'over sampling, SMOTENC, qui gère aussi bien les données numériques que catégorielles.

In [ ]:
dfmodel.columns

In [ ]:
Y = dfmodel['grav']
X = dfmodel.drop(['grav'],axis = 1)

In [ ]:
X.isnull().sum()

In [ ]:
X['an_nais'].mean(),X['an_nais'].median()

On choisit de compléter les 3007 valeurs manquantes d'années de naissance par la médiane des années de naissance car elle est moins sensible aux valeurs aberrantes que la moyenne. 

In [ ]:
median = X['an_nais'].median()
X['an_nais'].fillna(median, inplace=True)

Etant donné que nous avons un dataset déséquilibré avec beaucoup moins de valeurs de gravité égales à 1 (fusion des blessés hospitalisés et ds tués) que de valeurs égales à 0 ou 2 (indemnes et blessés légers), nous allons utiliser une méthode pour rééquilibrer le dataset appelé SMOTE NC. Le SMOTE, acronyme pour Synthetic Minority Oversampling TEchnique, est une méthode de suréchantillonnage des observations minoritaires. Il consiste à générer de nouveaux individus minoritaires qui ressemblent aux autres, sans être strictement identiques. Cela permet de densifier de façon homogène la population d’individus minoritaires. Cependant, le SMOTE classique ne permet pas de traiter les variables catégorielles. Pour traiter des données mixtes (mixed data), c’est-à-dire contenant à la fois des variables numériques et catégorielles, il faut utiliser une variante : le SMOTE-NC, pour SMOTE-Nominal Continuous.

In [ ]:
var_categoriques = [0,1,3,4,5,6,7,8,10,11,12] #indices des colonnes de features catégoriques
smNC = SMOTENC(categorical_features=var_categoriques)
XsmotedNC, YsmotedNC = smNC.fit_resample(X,Y)

In [ ]:
XsmotedNC.shape

In [ ]:
XsmotedNC.head(2)

In [ ]:
labels_reb, counts_reb = np.unique(YsmotedNC, return_counts=True)
plt.bar(labels_reb, counts_reb)
plt.gca().set_xticks(labels_reb)
plt.title('distribution des classes de gravité')
plt.show()

Nous avons bien ré-équilibré les classes comme nous le voulions. Ainsi, l'algorithme ne pourra plus prédire seulement la classe majoritaire afin d'assurer une bonne précision.

In [ ]:
features = ['place','sexe','secu1','catv','obsm','choc','manv','motor','lum','int','atm'] # features catégoriques qu'on veut encoder

In [ ]:
XsmotedNC2 = pd.get_dummies(XsmotedNC[features].astype(str)) # nous les encodons en indicatrices

In [ ]:
XsmotedNC2['an_nais'] = XsmotedNC['an_nais'] # nous rajoutons les variables numériques à notre dataset
XsmotedNC2['vma'] = XsmotedNC['vma']
XsmotedNC2['lat2'] = XsmotedNC['lat2']
XsmotedNC2['long2'] = XsmotedNC['long2']

In [ ]:
XsmotedNC2

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(XsmotedNC2,YsmotedNC)

### Premier modèle : Random Forest 

In [ ]:
model_rf = RandomForestClassifier(n_estimators=100, 
                                  max_depth=8)
#nous testons un premier modèle de Random Forest avec des paramètres classiques.

In [ ]:
model_rf.fit(Xtrain, Ytrain)
Ypredrf = model_rf.predict(Xtest)
test_acc = accuracy_score(Ytest, Ypredrf) 
# la métrique de précision aura dans toute la suite un sens car nos classes sont équilibrées. 
# Il faut donc faire mieux que 33% pour être meilleur que le hasard.
accu_rf = round(test_acc*100,1) 

In [ ]:
accu_rf

Pour visualiser la performance de notre modèle, nous utiliserons une matrice de confusion. 
La matrice de confusion est en quelque sorte un résumé des résultats de prédiction pour un problème particulier de classification. Elle compare les données réelles pour une variable cible à celles prédites par un modèle. Les prédictions justes et fausses sont révélées et réparties par classe, ce qui permet de les comparer avec des valeurs définies.
D'une façon générale, ces matrices excellent dans l'analyse rapide des données statistiques et de la simplification du déchiffrement des résultats par le biais de la Data Vizualisation. Ainsi, elles permettent d'apprécier les performances d'un modèle et d'identifier les tendances qui peuvent permettre d'en modifier les paramètres.

In [ ]:
cmrf = confusion_matrix(Ytest, Ypredrf,labels=model_rf.classes_)
disprf = ConfusionMatrixDisplay(cmrf,display_labels=model_rf.classes_)
disprf.plot()
plt.grid(False)
plt.title('Confusion matrix with Random Forest accuracy = '+str(accu_rf)+'%')
plt.show()

On constate que la deuxième classe est très mal prédite. Nous utilisons maintenant une RandomSearch pour commencer à optimiser les hyperparamètres du modèle.

In [ ]:
params_rf = {
    'n_estimators': [50,100,200],
    'max_depth': [3,5,7,9],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10]
}
#nous créons notre grille d'hyperparamètres.

In [ ]:
base_rfmodel = RandomForestClassifier()

In [ ]:
folds = 3
param_comb = 7
# nous faisons de la 3-fold cross validation et nous testons 7 combinaisons d'hyperparamètres dans la RandomSearch.

In [ ]:
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state =42)

random_search_rf = RandomizedSearchCV(base_rfmodel, param_distributions=params_rf, 
                                   n_iter=param_comb,scoring='accuracy', n_jobs=-1,
                                   cv=skf.split(Xtrain,Ytrain), verbose=3, random_state=42)

random_search_rf.fit(Xtrain,Ytrain)
# nous faisons de la random_search en cross-validation avec un paramètre de folds déjà défini plus haut. 

In [ ]:
print(random_search_rf.cv_results_)

In [ ]:
print(random_search_rf.best_estimator_)
#renvoie les modèle qui a obtenu la meilleure précision

In [ ]:
print(random_search_rf.best_params_)

In [ ]:
rf_best = random_search_rf.best_estimator_ # on utilise le meilleur modèle de notre RandomSearch
rf_best.fit(Xtrain,Ytrain)

Ypredrf_best = rf_best.predict(Xtest)
test_acc_rf_best = accuracy_score(Ytest, Ypredrf_best)
accu_rf_best = round(test_acc_rf_best,3)*100

cmrf_best = confusion_matrix(Ytest, Ypredrf_best,labels=rf_best.classes_)
disprf_best = ConfusionMatrixDisplay(cmrf_best,display_labels=rf_best.classes_)
disprf_best.plot()
plt.title('Confusion matrix with tuned RF after random search  : accuracy = '+str(accu_rf_best)+'%')
plt.grid(False)
plt.show()

In [ ]:
# Affinage avec GridSearch 

RandomSearch permet de tester aléatoirement des combinaisons pour trouver des hyperparamètres. Ainsi, cela nous a permis d'avoir une vision générale et approximative des hyperparamètres. Maintenant, on peut utiliser un Gridsearch qui teste toutes les combinaisons qu'on lui indique afin d'optimiser et d'affiner les hyperparamètres. 

In [ ]:
params_grid_rf = {
    'n_estimators': [150,175,200],
    'max_depth': [8,9,10,11],
    'min_samples_leaf': [1, 2,3],
    'min_samples_split': [1,2,3,4]
}
#nous créons notre grille d'hyperparamètres affinée pour le GridSearch.

In [ ]:
base_rfmodel = RandomForestClassifier()

In [ ]:
folds = 3

# nous faisons de la 3-fold cross validation 

In [ ]:
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state =42)

grid_search_rf = GridSearchCV(base_rfmodel, param_grid=params_grid_rf, scoring='accuracy', n_jobs=-1,
                                   cv=skf.split(Xtrain,Ytrain), verbose=3)

grid_search_rf.fit(Xtrain,Ytrain)
# nous faisons de la random_search en cross-validation avec un paramètre de folds déjà défini plus haut. 

In [ ]:
#print(grid_search_rf.cv_results_)

In [ ]:
print(grid_search_rf.best_estimator_)
#renvoie les modèle qui a obtenu la meilleure précision

In [ ]:
print(grid_search_rf.best_params_)

In [ ]:
rf_best = grid_search_rf.best_estimator_ # on utilise le meilleur modèle de notre RandomSearch
rf_best.fit(Xtrain,Ytrain)

Ypredrf_best = rf_best.predict(Xtest)
test_acc_rf_best = accuracy_score(Ytest, Ypredrf_best)
accu_rf_best = round(test_acc_rf_best,3)*100

cmrf_best = confusion_matrix(Ytest, Ypredrf_best,labels=rf_best.classes_)
disprf_best = ConfusionMatrixDisplay(cmrf_best,display_labels=rf_best.classes_)
disprf_best.plot()
plt.title('Confusion matrix with tuned RF after grid search  : accuracy = '+str(accu_rf_best)+'%')
plt.grid(False)
plt.show()

Le gridsearch était plutôt long (10 minutes) et le gain négligeable, il est raisonnable de passer à un autre modèle que les random forests

### Deuxième modèle : XGBoost

XGBoost est un algorithme de machine learning appartenant à la classe des alogrithmes utilisant le Gradient Boosting. Cela signifie qu'il ne construit pas une forêt d'arbres de décision indépendants pour prédire mais au contraire qu'il construit chaque nouvel arbre de décision en utilisant les erreurs commises par les précédents. L'apprentissage est donc séquentiel et non plus en parallèle.

XGBoost est notamment très utilisé pour les compétitions Kaggle et notamment par les vainqueurs, il domine actuellement les autres algorithmes en termes de performance et de vitesse.

In [ ]:
model_xgb = XGBClassifier(objective='multi:softprob',learning_rate=0.3,max_depth=4,subsample=0.75,n_estimators=150) # nous testons un premier modèle
# avec des paramètres pris au hasard

In [ ]:
model_xgb.fit(Xtrain,Ytrain)

Ypredxgb = model_xgb.predict(Xtest)
test_acc_xgb = accuracy_score(Ytest, Ypredxgb)
accu_xgb = round(test_acc_xgb*100,1)

In [ ]:
cmxgb = confusion_matrix(Ytest, Ypredxgb,labels=model_xgb.classes_)
dispxgb = ConfusionMatrixDisplay(cmxgb,display_labels=model_xgb.classes_)
dispxgb.plot()
plt.title('Confusion matrix with XGBOOST accuracy = '+str(accu_xgb)+'%')
plt.grid(False)
plt.show()

In [ ]:
params_xgb = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'learning_rate' : [0.1,0.3,0.5,0.7],
        'n_estimators': [75,100,150,200]
        }
# nous créons notre grille d'hyperparamètres à tester

In [ ]:
xgb = XGBClassifier(objective='multi:softprob',silent=True, nthread=1)

In [ ]:
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state =42,)

random_search_xgb = RandomizedSearchCV(xgb, param_distributions=params_xgb, 
                                   n_iter=param_comb,scoring='accuracy',
                                   cv=skf.split(Xtrain,Ytrain), verbose=3, random_state=42)

random_search_xgb.fit(Xtrain,Ytrain)
# nous faisons de la random_search en cross-validation avec un paramètre de folds déjà défini plus haut. 

In [ ]:
print(random_search_xgb.cv_results_)

In [ ]:
print(random_search_xgb.best_estimator_)

In [ ]:
print(random_search_xgb.best_params_)

In [ ]:
xgb_best = random_search_xgb.best_estimator_
xgb_best.fit(Xtrain,Ytrain)

Ypredxgb_best = xgb_best.predict(Xtest)
test_acc_xgb_best = accuracy_score(Ytest, Ypredxgb_best)
accu_xgb_best = round(test_acc_xgb_best,3)*100

cmxgb_best = confusion_matrix(Ytest, Ypredxgb_best,labels=xgb_best.classes_)
dispxgb_best = ConfusionMatrixDisplay(cmxgb_best,display_labels=xgb_best.classes_)
dispxgb_best.plot()
plt.grid(False)
plt.title('Confusion matrix with tuned XGBOOST after random search  : accuracy = '+str(accu_xgb_best)+'%')
plt.show()

On remarque que les classes 0 (indemne) et 1 (tué et bléssé hospitalisé) sont plutôt bien prédites par notre modèle. A l'inverse, la classe 2 (bléssé léger) semble être un peu moins évidente à prédire pour notre modèle.

Pour info, 2 bons modèles : gamma=1, learning_rate=0.7, max_depth=5, min_child_weight=10, n_estimators=150, subsample=1.0
et gamma=1.5, learning_rate=0.7, max_depth=5, min_child_weight=10, n_estimators=200, subsample=0.8

In [ ]:
# Affinage avec GridSearch 

In [ ]:
params_grid_rf = {
    'n_estimators': [150,175,200],
    'max_depth': [8,9,10,11],
    'min_samples_leaf': [1, 2,3],
    'min_samples_split': [1,2,3,4]
}
#nous créons notre grille d'hyperparamètres affinée pour le GridSearch.

In [ ]:
base_rfmodel = RandomForestClassifier()

folds = 3
# nous faisons de la 3-fold cross validation 

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state =42)

grid_search_rf = GridSearchCV(base_rfmodel, param_grid=params_grid_rf, scoring='accuracy', n_jobs=-1,
                                   cv=skf.split(Xtrain,Ytrain), verbose=3)

grid_search_rf.fit(Xtrain,Ytrain)
# nous faisons de la random_search en cross-validation avec un paramètre de folds déjà défini plus haut. 

print(grid_search_rf.cv_results_)
print(grid_search_rf.best_estimator_)
#renvoie les modèle qui a obtenu la meilleure précision
print(grid_search_rf.best_params_)

rf_best = grid_search_rf.best_estimator_ # on utilise le meilleur modèle de notre RandomSearch
rf_best.fit(Xtrain,Ytrain)

Ypredrf_best = rf_best.predict(Xtest)
test_acc_rf_best = accuracy_score(Ytest, Ypredrf_best)
accu_rf_best = round(test_acc_rf_best,3)*100

cmrf_best = confusion_matrix(Ytest, Ypredrf_best,labels=rf_best.classes_)
disprf_best = ConfusionMatrixDisplay(cmrf_best,display_labels=rf_best.classes_)
disprf_best.plot()
plt.title('Confusion matrix with tuned RF after grid search  : accuracy = '+str(accu_rf_best)+'%')
plt.grid(False)
plt.show()

In [ ]:
base_rfmodel = RandomForestClassifier()

In [ ]:
folds = 3
# nous faisons de la 3-fold cross validation 

In [ ]:
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state =42)

grid_search_rf = GridSearchCV(base_rfmodel, param_grid=params_grid_rf, scoring='accuracy', n_jobs=-1,
                                   cv=skf.split(Xtrain,Ytrain), verbose=3)

grid_search_rf.fit(Xtrain,Ytrain)
# nous faisons de la random_search en cross-validation avec un paramètre de folds déjà défini plus haut. 

In [ ]:
print(grid_search_rf.cv_results_)

print(grid_search_rf.best_estimator_)

#renvoie les modèle qui a obtenu la meilleure précision
print(grid_search_rf.best_params_)

In [ ]:
rf_best = grid_search_rf.best_estimator_ # on utilise le meilleur modèle de notre RandomSearch
rf_best.fit(Xtrain,Ytrain)

Ypredrf_best = rf_best.predict(Xtest)
test_acc_rf_best = accuracy_score(Ytest, Ypredrf_best)
accu_rf_best = round(test_acc_rf_best,3)*100

cmrf_best = confusion_matrix(Ytest, Ypredrf_best,labels=rf_best.classes_)
disprf_best = ConfusionMatrixDisplay(cmrf_best,display_labels=rf_best.classes_)
disprf_best.plot()
plt.title('Confusion matrix with tuned RF after grid search  : accuracy = '+str(accu_rf_best)+'%')
plt.grid(False)
plt.show()

### Troisième modèle : SVM

L’une des méthodes de Machine Learning les plus utilisées en classification est les SVM (Support Vector Machines). Il s’agit de trouver, dans un système de projection adéquat (noyau ou kernel), les paramètres de l’hyperplan (en fait d’un hyperplan à marges maximales) séparant les classes de données:

In [ ]:
model_svm = 

## FEATURE IMPORTANCE & SHAP

On veut maintenant savoir sur quelles informations se base notre modèle pour effectuer ses prédéctions. En d'autres termes, quelles sont les variables les plus influentes pour prédire la gravité d'un accident de voiture ?

Pour cela, nous allons dans un premier temps utiliser la méthode de feature importance de scikit-learn puis la bibliothèque SHAP.

In [ ]:
xgb_best.feature_importances_

In [ ]:
plt.barh(Xtrain.columns, xgb_best.feature_importances_)

In [ ]:
xgboost.plot_importance(xgb_best,max_num_features=15)
plt.title("Feature importance")
plt.show()

In [ ]:
!pip install shap

In [ ]:
import shap

In [ ]:
explainer = shap.Explainer(xgb_best, Xtrain)
shap_values = explainer(Xtrain)

In [ ]:
shap_values

In [ ]:
shap.summary_plot(shap_values.data, Xtrain, plot_type="bar")